In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pybaseball
from pybaseball import statcast
pybaseball.cache.enable()

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [4]:
%%time
# .iloc[::-1].reset_index(drop=True): reverse the row order because the default of row 1 is the most recent instance
df = statcast(start_dt='2023-03-30',end_dt='2023-10-01').iloc[::-1].reset_index(drop=True)
df.shape

This is a large query, it may take a moment to complete


100%|██████████| 186/186 [17:14<00:00,  5.56s/it]


CPU times: total: 5min 10s
Wall time: 17min 49s


(717945, 113)

In [48]:
# check 
check_n_home_game = df.groupby(['home_team'])['game_pk'].nunique()
print(check_n_home_game.unique())
print(f'number of teams in the query: {len(check_n_home_game)}')

[81]
number of teams in the query: 30


In [49]:
# check row order 
print(df[['game_pk','game_date','inning','inning_topbot']].head())
print(df[['game_pk','game_date','inning','inning_topbot']].tail())

   game_pk  game_date  inning inning_topbot
0   718767 2023-03-30       1           Top
1   718767 2023-03-30       1           Top
2   718767 2023-03-30       1           Top
3   718767 2023-03-30       1           Top
4   718767 2023-03-30       1           Top
        game_pk  game_date  inning inning_topbot
717940   716367 2023-10-01       9           Bot
717941   716367 2023-10-01       9           Bot
717942   716367 2023-10-01       9           Bot
717943   716367 2023-10-01       9           Bot
717944   716367 2023-10-01       9           Bot


In [138]:
batting_stats["Date"] = pd.to_datetime(batting_stats["Date"])
batting_stats.head()

,Name,Age,#days,Lev,Date,Tm,,Opp,G,PA,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
0,CJ Abrams,22,595,Maj-NL,2023-03-30,Washington,NaN,Atlanta,1,4,...,0,0,0,0,0,0.000,0.000,0.000,0.000,682928
1,Jos\xc3\xa9 Abreu,36,595,Maj-AL,2023-03-30,Houston,NaN,Chicago,1,4,...,0,0,0,0,0,0.250,0.250,0.250,0.500,547989
2,Ronald Acu\xc3\xb1a Jr.,25,595,Maj-NL,2023-03-30,Atlanta,@,Washington,1,6,...,0,0,0,1,0,0.333,0.333,0.333,0.667,660670
3,Willy Adames,27,595,Maj-NL,2023-03-30,Milwaukee,@,Chicago,1,4,...,0,0,1,0,0,0.333,0.500,0.333,0.833,642715
4,Jes\xc3\xbas Aguilar,33,595,Maj-AL,2023-03-30,Oakland,NaN,Los Angeles,1,4,...,0,0,0,0,0,0.000,0.250,0.000,0.250,542583


In [150]:
#Marcus added
def get_batter_stats(data, batter_stats = ["Date","G", "PA", "AB", "R", "H", "2B", "3B", "HR", "RBI", "BB", "IBB", "SO", "HBP", "SH", "SF", "GDP", "SB", "CS", "BA", "OBP", "SLG", "OPS","mlbID"]):
    batting_stats = pd.read_csv("batter2023_03_30.csv")
    batting_stats["Date"] = pd.to_datetime(batting_stats["Date"])

    
    bat_clean = batting_stats.loc[:,batter_stats]


    batters = ["away_b1", "away_b2", "away_b3", "away_b4", "away_b5", "away_b6", "away_b7", "away_b8", "away_b9", "home_b1", "home_b2", "home_b3", "home_b4", "home_b5", "home_b6", "home_b7", "home_b8", "home_b9"]
    for batter in batters:
        data = data.merge(bat_clean, how="left", left_on= ["date",batter], right_on=["Date","mlbID"], suffixes=("","_"+batter))

            
    
    data = data.rename(columns = lambda col: col + "_away_b1" if col in batter_stats else col)
    for batter in batters:
        data = data.drop(columns=["Date_"+batter, "mlbID_" + batter])
    return data

In [151]:
data = get_batter_stats(data_with_batters)
data.to_csv('data_with_batter_stats.csv', index=True)

In [52]:
# list out what kind of data we need for each game
cols = ['home_result','game_pk','date','away_team','home_team','away_score','home_score','away_starting_pitcher','home_starting_pitcher']

# add batters 1-9 for both away and home
for i in ['away','home']:
    for j in range(1,10):
        cols.append(f'{i}_b{j}')

# colnames for pitcher metrics: 'sp_era','sp_k9','sp_bb','bp_era','bp_k9','bp_bb'
# sp: starting pitcher; bp: bullpen; bb: pitcher's walk rate
# pitcher metrics will be added later
# batter metrics will be added later as well

print(cols[:10])
print(cols[10:20])
print(cols[20:])

['home_result', 'game_pk', 'date', 'away_team', 'home_team', 'away_score', 'home_score', 'away_starting_pitcher', 'home_starting_pitcher', 'away_b1']
['away_b2', 'away_b3', 'away_b4', 'away_b5', 'away_b6', 'away_b7', 'away_b8', 'away_b9', 'home_b1', 'home_b2']
['home_b3', 'home_b4', 'home_b5', 'home_b6', 'home_b7', 'home_b8', 'home_b9']


In [53]:
data_without_batters = df.groupby('game_pk').apply(lambda group: pd.Series({
    'home_result': None,
    'date': group['game_date'].iloc[0],  # only one corresponding value
    'away_team': group['away_team'].iloc[0],  # only one corresponding value
    'home_team': group['home_team'].iloc[0],  # only one corresponding value
    'away_score': group['away_score'].max(),
    'home_score': group['home_score'].max(),
    'away_starting_pitcher': group.loc[(group['inning'] == 1) & (group['inning_topbot'] == 'Bot'), 'pitcher'].iloc[0],
    'home_starting_pitcher': group.loc[(group['inning'] == 1) & (group['inning_topbot'] == 'Top'), 'pitcher'].iloc[0],
})).sort_values(by='date', ascending=True).reset_index()
data_without_batters.head()

C:\Users\Welcome\AppData\Local\Temp\ipykernel_21344\1718263223.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_without_batters = df.groupby('game_pk').apply(lambda group: pd.Series({


,game_pk,home_result,date,away_team,home_team,away_score,home_score,away_starting_pitcher,home_starting_pitcher
0,718782,None,2023-03-30,BAL,BOS,10,9,502043,446372
1,718781,None,2023-03-30,SF,NYY,0,5,657277,543037
2,718767,None,2023-03-30,CLE,SEA,0,3,669456,622491
3,718768,None,2023-03-30,CWS,HOU,3,2,656302,664285
4,718770,None,2023-03-30,AZ,LAD,2,8,668678,628711


In [54]:
data_without_batters['home_result'] = np.where(data_without_batters['home_score'] > data_without_batters['away_score'], 'W', 'L')
data_without_batters.head()

,game_pk,home_result,date,away_team,home_team,away_score,home_score,away_starting_pitcher,home_starting_pitcher
0,718782,L,2023-03-30,BAL,BOS,10,9,502043,446372
1,718781,W,2023-03-30,SF,NYY,0,5,657277,543037
2,718767,W,2023-03-30,CLE,SEA,0,3,669456,622491
3,718768,L,2023-03-30,CWS,HOU,3,2,656302,664285
4,718770,W,2023-03-30,AZ,LAD,2,8,668678,628711


In [55]:
def get_away_batting_order(group):
    away_batters = group.loc[group['inning_topbot'] == 'Top', 'batter'].unique()[:9]
    away_b_dict = {f'away_b{i+1}':away_batters[i] for i in range(9)}
    return pd.Series(away_b_dict)

def get_home_batting_order(group):
    home_batters = group.loc[group['inning_topbot'] == 'Bot', 'batter'].unique()[:9]
    home_b_dict = {f'home_b{i+1}':home_batters[i] for i in range(9)}
    return pd.Series(home_b_dict)


In [56]:
away_bs = df.groupby('game_pk').apply(get_away_batting_order)
home_bs = df.groupby('game_pk').apply(get_home_batting_order)

C:\Users\Welcome\AppData\Local\Temp\ipykernel_21344\1153644726.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  away_bs = df.groupby('game_pk').apply(get_away_batting_order)
C:\Users\Welcome\AppData\Local\Temp\ipykernel_21344\1153644726.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  home_bs = df.groupby('game_pk').apply(get_home_batting_order)


In [57]:
data_with_batters = data_without_batters.merge(away_bs, on='game_pk').merge(home_bs, on='game_pk')
print(data_with_batters.shape)
data_with_batters.head()

(2430, 27)


,game_pk,home_result,date,away_team,home_team,away_score,home_score,away_starting_pitcher,home_starting_pitcher,away_b1,...,away_b9,home_b1,home_b2,home_b3,home_b4,home_b5,home_b6,home_b7,home_b8,home_b9
0,718782,L,2023-03-30,BAL,BOS,10,9,502043,446372,656775,...,622761,657077,646240,457759,807799,594807,671213,624414,624512,571771
1,718781,W,2023-03-30,SF,NYY,0,5,657277,543037,664774,...,547379,518934,592450,519203,519317,518626,650402,665828,624431,683011
2,718767,W,2023-03-30,CLE,SEA,0,3,669456,622491,680757,...,664702,677594,543939,664034,606192,663728,553993,672284,600303,641487
3,718768,L,2023-03-30,CWS,HOU,3,2,656302,664285,641313,...,663853,665161,608324,670541,547989,663656,673237,676694,455117,643289
4,718770,W,2023-03-30,AZ,LAD,2,8,668678,628711,641786,...,664983,605141,518692,669257,571970,502110,444482,678246,681546,500743


In [67]:
data_with_batters.to_csv('data_without_metric.csv', index=True)